In [3]:
!pip install -U -q PyDrive

from keras import backend as K
import scipy.io as sio
from keras import Sequential
from keras.layers.core import Dense, Activation, Dropout,Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D, BatchNormalization, SimpleRNN, LSTM, GRU, Bidirectional
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
import time
from sklearn import svm 
from sklearn import metrics
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score
from IPython.display import Image
import pandas as pd
from keras.utils import np_utils
import pickle
from keras.callbacks import ModelCheckpoint
import sys 
import re
import heapq
import random
from IPython.display import Image

# chequeamos que funcione GPU 
K.tensorflow_backend._get_available_gpus()

/bin/sh: pip: command not found


[]

In [0]:
!wget https://www.dropbox.com/s/nlyqf5uulx6sr8d/ArchivosParte1.zip
!unzip ArchivosParte1.zip

In [0]:
df = pd.read_csv('international-airline-passengers.csv', sep=',')

In [0]:
df.head(5)

In [0]:
modelRNN = Sequential()
modelRNN.add(SimpleRNN(5,input_dim=1,input_length=4,return_sequences=False))
modelRNN.summary()

Total de parametros: 35  <br>
Correspondiente a:<br>

Wxh = 5 x 1 = 5 params   (entrada) <br>
Whh = 5 x 5 = 25 params(hidden layer) <br> 
Why = 5 x 1 = 5 params (salida) <br>

Total parametros = 35 

In [0]:
# con 4 dimensiones 
modelRNN = Sequential()
modelRNN.add(SimpleRNN(4,input_dim=1,input_length=4,return_sequences=False))
modelRNN.summary()

Ahora si modificamos la dimension a 4, el total de parametros baja a 24 explicado por: <br>

Wxh = 4 x 1   (entrada)<br>
Whh = 4 x 4 = 16 (hidden layer)<br> 
Why = 4 x 1 (salida) <br>

Total parametros = 24

Funciones Auxiliares Airline prediction (AirlinePrediction.py y AirlineUtils.py)  <br> 

In [0]:
#!/usr/bin/env python
import pandas
import numpy
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)


def readAirlineData(history):
    # load the dataset
    dataframe = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = 96 #Enero 1949 a Diciembre 1956
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    # reshape into X=t and Y=t+1
    trainX, trainY = create_dataset(train, history)
    testX, testY = create_dataset(test, history)
    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
    testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1],1))
    return trainX, trainY, testX, testY, scaler, dataset


def displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history):
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    
    #return trainScore, testScore
    # shift train predictions for plotting
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[history:len(trainPredict)+history, :] = trainPredict

    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(history*2)+1:len(dataset)-1, :] = testPredict

    # plot baseline and predictions
    plt.plot(scaler.inverse_transform(dataset))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()
    
    return (trainScore, testScore)

In [0]:
#set sequence length
history=4 

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()
modelRNN.add(SimpleRNN(5,input_dim=1,input_length=history, return_sequences=False))
modelRNN.add(Dense(1))

RMSE_results = []
#Train model
# ejecutamos 5 veces 
for i in range(5):

  modelRNN.compile(loss='mean_squared_error', optimizer='adam')
  modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

  # Make predictions
  trainPredict = modelRNN.predict(trainX)
  testPredict = modelRNN.predict(testX)

  #Display results
  RMSE_results.append(displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history))
  
  # reseteamos los pesos de la red 
  modelRNN = Sequential()
  modelRNN.add(SimpleRNN(5,input_dim=1,input_length=history, return_sequences=False))
  modelRNN.add(Dense(1))

RMSE_results

En 5 iteraciones cambian los resultados de RMSE en set de train y testing:  <br>
Train RMSE | Test RMSE <br>
(28.601777834802732, 87.99367348269288), <br>
 (21.060396047062273, 65.64185217447363), <br>
 (20.541622444370542, 64.23740290571874), <br>
 (20.304950774496767, 68.34587708863847), <br>
 (20.49618758870263, 72.14143507462187) <br> 
 
 Esto se puede deber a que los pesos iniciales son aleatorios y esto puede hacer que cambien los resultados de la prediccion. 

### Cambiando el largo de la secuencia de entrada (history), como afecta precision?

In [0]:
#set sequence length (cambiar y ver como afecta precision)
input_length=4

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()
modelRNN.add(SimpleRNN(5,input_dim=1,input_length=input_length, return_sequences=False))
modelRNN.add(Dense(1))

#Train model
modelRNN.compile(loss='mean_squared_error', optimizer='adam')
modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

# Make predictions
trainPredict = modelRNN.predict(trainX)
testPredict = modelRNN.predict(testX)

#Display results
displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history)


Length = 4 , RMSE = (22.13, 47.06)         <br>
Length = 8 , RMSE =  (23.69, 61.11)       <br>
Length = 12 , RMSE = (34.36, 111.65)      <br>

Al aumentar el largo de la entrada tiene peor precision, mayor RMSE.

Esto se puede deber a que al aumentar el largo de la entrada tiene menos datos de entrenamiento para mejorar la precision del modelo. 



In [0]:
#set sequence length (cambiar y ver como afecta precision)
history=4

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()
modelRNN.add(SimpleRNN(5,input_dim=1,input_length=history, return_sequences=False))

# agreamos nueva capa densa 100 unidades  
modelRNN.add(Dense(100))

# capa de salida dimension = 1 
modelRNN.add(Dense(1))

#Train model
modelRNN.compile(loss='mean_squared_error', optimizer='adam')
modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

# Make predictions
trainPredict = modelRNN.predict(trainX)
testPredict = modelRNN.predict(testX)

#Display results
displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history)


In [0]:
#set sequence length (cambiar y ver como afecta precision)
history=4

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()
modelRNN.add(SimpleRNN(5,input_dim=1,input_length=history, return_sequences=False))

# agregamos capa DROPOUT 
modelRNN.add(Dropout(0.5))

# capa de salida 
modelRNN.add(Dense(1))

#Train model
modelRNN.compile(loss='mean_squared_error', optimizer='adam')
modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

# Make predictions
trainPredict = modelRNN.predict(trainX)
testPredict = modelRNN.predict(testX)

#Display results
displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history)

Al agregar DROP OUT el RMSE aumenta considerablemente, sobre todo en set de testing... es decir empeora la precision del modelo al agregar DROPOUT. En el caso anterior el RMSE de test fue de 47 y ahora aumento a 94. 
Esto es porque el modelo queda con menos neuronas para modificar parametros y entrenar el modelo. 
Ademas como son tan pocas capas pierde muchos parametros.

In [0]:
#set sequence length (cambiar y ver como afecta precision)
history=4

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()

# Aqui cambiamos por LSTM 
modelRNN.add(LSTM(5,input_dim=1,input_length=history, return_sequences=False))

# capa de salida 
modelRNN.add(Dense(1))

#Train model
modelRNN.compile(loss='mean_squared_error', optimizer='adam')
modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

# Make predictions
trainPredict = modelRNN.predict(trainX)
testPredict = modelRNN.predict(testX)

#Display results
displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history)

**En el caso anterior el RMSE de test fue de 47 y ahora AUMENTO a 59.68** deberia mejorar pero como son tan pocos datos no alcanza a mejorar su prediccion. Lo que hace LSTM es solucionar el problema de que los weights se hagan muy pequeños o muy grandes al aplicar el gradiente. Pero en este caso como son tan pocas capas y datos este problema no alcanza a suceder. 

**En LSTM se usaron 146 parametros comparados con la SimpleRNN que uso tan solo 41 parametros. **

In [0]:
modelRNN.summary()

### GRU

In [0]:
#set sequence length (cambiar y ver como afecta precision)
history=4

#read training and test sets
trainX, trainY, testX, testY, scaler, dataset=readAirlineData(history)

# create and fit the LSTM network
modelRNN = Sequential()

# Aqui cambiamos por GRU 
modelRNN.add(GRU(5,input_dim=1,input_length=history, return_sequences=False))

# Capa de salida 
modelRNN.add(Dense(1))

#Train model
modelRNN.compile(loss='mean_squared_error', optimizer='adam')
modelRNN.fit(trainX, trainY, nb_epoch=150, batch_size=5, verbose=2)

# Make predictions
trainPredict = modelRNN.predict(trainX)
testPredict = modelRNN.predict(testX)

#Display results
displayResult(dataset, trainPredict, trainY, testPredict, testY, scaler, history)

**En el caso anterior el RMSE de test fue de 47 y ahora AUMENTO a 56.66** deberia mejorar pero como son tan pocos datos no alcanza a mejorar su prediccion. <br> 

Con GRU se utilizaron 111 parametros comparados con los 41 que se usaron en la SimpleRNN. 

In [0]:
modelRNN.summary()

## DOCUMENTS MODELING

En este modelo utilizaremos como datos de entrenamiento los caracteres y buscaremos predecir el caracter siguiente. 

Funciones auxiliares

In [4]:
def vocabulary_obtention(data):
  chars = list(data.split())
  char_to_int = dict((c, i) for i, c in enumerate(chars))
  int_to_char = dict((i, c) for i, c in enumerate(chars))
  n_chars = len(data)
  n_vocab = len(chars)
  return char_to_int, int_to_char,n_chars, n_vocab 

def sequence_arrays(data, seq_length, n_chars):
  X = []
  Y = []
  for i in range(0, n_chars - seq_length):
    seq_in = data.split()[i:i + seq_length]
    seq_out = data.split()[i + seq_length]
    X.append([char_to_int[char] for char in seq_in if char in char_to_int])
    Y.append(char_to_int[seq_out]) 
  n_patterns = len(X)
  #n_patterns = 4573338
  return X, Y, n_patterns 

def feature_normalization(X, n_vocab, n_patterns):
  X = np.reshape(X, (n_patterns, seq_length, 1))
  X = X / float(n_vocab)
  return X 
 
def save_to_drive(filepath):
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': filepath})
  uploaded.SetContentFile(filepath)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

def generate_text(X,char_to_int,int_to_char,model, n_vocab, length_text):  
  init = np.random.randint(0, len(X)-1)
  phrase = X[init]

  print ("Primer parrafo")
  print ("\"",''.join([int_to_char[value] for value in phrase]), "\"")

  # RNN genera caracteres escogidos  
  for i in range(length_text):
    x = np.reshape(phrase, (1, len(phrase), 1))
    x = x/float(n_vocab)
    
    # predice un embedding de la palabra 
    prediction = model.predict(x, verbose=0)
    
    
    # elige al azar uno de los mas altos 
    index = np.argmax(prediction)
    
    if index in int_to_char:
      # pasar de index a caracter   
      result = int_to_char[index]
      seq_in = [int_to_char[value] for value in phrase]
      sys.stdout.write(result)
      phrase.append(index)
      phrase = phrase[1:len(phrase)]

  print("\nFINISHED.")


Procesamiento del texto: 
- creamos un diccionario con cada letra y su index
- diccionario con key letra y value el index y vice versa para hacer la busqueda despues

In [5]:
data = open('shakespeare_input.txt', 'r').read()

# pasamos todo a minusculas 
data = data.lower()

In [6]:
char_to_int, int_to_char,n_chars, n_vocab = vocabulary_obtention(data)

In [7]:
n_chars

4573338

In [8]:
print('total caracteres: {}'.format(n_chars))
print('largo vocabulario: {}'.format(n_vocab))

total caracteres: 4573338
largo vocabulario: 832301


In [ ]:
X_array, Y_array, n_patterns = sequence_arrays(data, seq_length = 100, n_chars = n_chars)

In [10]:
print('total patterns: {}'.format(n_patterns))

total patterns: 4573338


Normalizamos por el largo del vocabulario y convertimos a one hot encoder el array Y: 

In [0]:
# este se cae en colab, hay que subir esta array de dropbox (abajo) 
X = feature_normalization(X_array, n_vocab, n_patterns)

y = np_utils.to_categorical(Y_array)

Cargamos array X de dropbox (Colab se cae en paso anterior...) 

In [10]:
!wget https://www.dropbox.com/s/vn3d90s6n4q7jpo/X.npy

--2018-05-30 00:23:41--  https://www.dropbox.com/s/vn3d90s6n4q7jpo/X.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/Fdnf2FpeGqA98CgJEroI4a0RWzS7uAZXsDngmEf8kfLHhxCoOv4DE8tVQkxDlvEU/file [following]
--2018-05-30 00:23:42--  https://dl.dropboxusercontent.com/content_link/Fdnf2FpeGqA98CgJEroI4a0RWzS7uAZXsDngmEf8kfLHhxCoOv4DE8tVQkxDlvEU/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.9.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.9.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3658590528 (3.4G) [application/octet-stream]
Saving to: ‘X.npy’

X.npy                68%[============>       ]   2.35G   137MB/s    eta 10s    X.npy               100%[=

In [13]:
!ls

datalab  shakespeare_input.txt	X.npy


In [0]:
X = np.load('X.npy')

Es un problema de clasificacion con 41 posibles clases (largo del vocabulario): <br> 

**LSTM Normal:** <br> 
- 2 LSTM y un DropOut con prob 0.15 
- Incluimos un clipvalue para que el gradiente se mueva dentro del rango -0.7 y 0.7
- En la ultima capa una softmax para predecir la siguiente clase 
- Optimizador SGD
- Batchsize de 500


**LSTM Bidireccional ** <br> 
- Capa LSTM Bidireccional 
- Drop out de 0.15 prob 
- clipvalue en rango -0.7 y 0.7 


**AGREGAR DIAGRAMA DE BLOQUES **

In [0]:
Image("diagrama_LSTM.png")

**LSTM NORMAL**

In [0]:
#cargar weights 
#files.upload()

# meta-params 
epochs = 12 
batch_size = 500

# clipvalue limites de gradiente (-0.7 - 0.7)
optimizer = optimizers.Adam(clipvalue = 0.7)

# capas del modelo 
model = Sequential()

model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))

#model.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(X.shape[1], X.shape[2])))

model.add(Dropout(0.15))

model.add(LSTM(256))

model.add(Dropout(0.15))

model.add(Dense(y.shape[1], activation='softmax'))

# load weights (cambiar dependiendo si queremos iniciar un modelo anterior) 
#path_weights = 'weights_best_RNN_P2_LSTM_11_epochs.hdf5'
#model.load_weights(path_weights)

model.compile(loss='categorical_crossentropy', optimizer= optimizer )
filepath="weights_best_RNN_P2_LSTM.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs= epochs, batch_size= batch_size, callbacks=callbacks_list)

**LSTM BIDIRECCIONAL**

Para LSTM Bidireccional consideramos <br> 
- 2 capas LSTM Bidireccionales 
- clipvalue en rango -0.7 y 0.7 
- batchsize de 500
- drop out con probabilidad de 0.15 despues de cada capa LSTM Bidireccional

In [25]:
!ls

datalab  shakespeare_input.txt	weights_best_RNN_P2_LSTM.hdf5  X.npy


In [0]:
#cargar weights 
#files.upload()

# meta-params 
epochs = 8 
batch_size = 500

# clipvalue limites de gradiente (-0.7 - 0.7)
optimizer = optimizers.Adam(clipvalue=0.7)

# capas del modelo 
model = Sequential()

model.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(X.shape[1], X.shape[2])))

model.add(Dropout(0.15))

model.add(Bidirectional(LSTM(256)))

model.add(Dropout(0.15))

model.add(Dense(y.shape[1], activation='softmax'))

# load weights (cambiar dependiendo si queremos iniciar un modelo anterior) 
#path_weights = 'weights_best_RNN_P2_LSTM_1_epochs.hdf5'
#model.load_weights(path_weights)

model.compile(loss='categorical_crossentropy', optimizer= optimizer )
filepath="weights_best_RNN_P2_LSTM_Bidireccional.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs= epochs, batch_size= batch_size, callbacks=callbacks_list)



Epoch 1/8
2439000/4573238 [==============>...............] - ETA: 1:19:38 - loss: 2.3473

In [17]:
#files.upload()
weights_file = "weights_best_RNN_P2_LSTM.hdf5"

# ESTO ES PARA GUARDAR EN DRIVE EL ARCHIVO LLAMADO weights_modelo_final.best.hdf5
save_to_drive(weights_file)

Uploaded file with ID 13FSBNVmEjPPYR5AIrsQDUxYwMs5dDPLb


In [16]:
!ls

datalab  shakespeare_input.txt	weights_best_RNN_P2_LSTM.hdf5  X.npy


Ahora veremos como nuestra RNN genera texto para LSTM NORMAL

In [44]:
# largo del texto que queremos generar 
#length_text = 500 

#generate_text(X_array, char_to_int, int_to_char, model, n_vocab, length_text)

Primer parrafo
" ld the cutting of my garments would serve the
turn, or the breaking of my spanish sword.

second lor "
d:
why, then the sea were a son to the sea,
and therefore i will be a single thought.

parolles:
a man and the sea will be so so the world,
and therefore i will be a state of thee.

cassio:
why, then the sea were a son of the sea,
and therefore the sense is a son of thee.

cassio:
why, then the sea were a son of the sea,
and therefore the sense is a son of thee.

cassio:
why, then the sea were a son of the sea,
and therefore the sense is a son of thee.

cassio:
why, then the sea were a son of th
FINISHED.


Vemos que repite muchas veces la frase "sea were a son of the sea" esto es porque no se está entrenando con palabras de adelante y de atras como lo haria la LSTM Bidireccional. 

Para LSTM BIDERECCIONAL 

In [24]:
# largo del texto que queremos generar 
length_text = 200 

generate_text(X_array, char_to_int, int_to_char, model, n_vocab, length_text)

Primer parrafo
" ion:
no doubt the murderous knife was dull and blunt
till it was whetted on thy stone-hard heart,
to "
 the common of the country of the courtesy.

cassio:
i will not be a man of the country of the courtesy.

cassio:
i will not be a man of the country of the court
of the court of the court of the court
FINISHED.
